<a href="https://colab.research.google.com/github/harrisontg57/NLP_Spring_2021/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Tim Harrison ID: 002-24-5948

Problem 1)

In [ ]:
import re
import sklearn
import nltk